In [1]:
"""
This program cleans data originating from:   'D:/Py4e/Beautiful Soup Tutorials/techwithtim tutorial/scraptethissite.py'
This program adds calculated columns
This program removes junk columns
This program exports a CSV to :    'D:/Py4e/Beautiful Soup Tutorials/techwithtim tutorial/teams.csv'

Webpage Scraped: https://www.scrapethissite.com/pages/forms/

Output used in Visualization in Tableau Public

"""
import requests
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.close("all")
from pathlib import Path
from bs4 import BeautifulSoup
from io import StringIO
import seaborn as sns

In [2]:
#Using Beautiful Soup to pull data from a website. A loop is used to paginate through the total of the table.

fresult = "Team Name,Year,Wins,Losses,OT Losses,Win%,Goals For,Goals Against,+/-"
for i in range(1,7):
    url = f'https://www.scrapethissite.com/pages/forms/?page_num={i}&per_page=100'
    response = requests.get(url)
    response = response.content
    soup = BeautifulSoup(response, 'html.parser')
    print(url)
    
    teamdata = soup.find_all("tr", class_="team")
    for string in teamdata:
        fresult = fresult + "\n" + (string.get_text(",",strip=True))

https://www.scrapethissite.com/pages/forms/?page_num=1&per_page=100
https://www.scrapethissite.com/pages/forms/?page_num=2&per_page=100
https://www.scrapethissite.com/pages/forms/?page_num=3&per_page=100
https://www.scrapethissite.com/pages/forms/?page_num=4&per_page=100
https://www.scrapethissite.com/pages/forms/?page_num=5&per_page=100
https://www.scrapethissite.com/pages/forms/?page_num=6&per_page=100


In [3]:
#Converting the table data from the scrape to a Pandas Dataframe

csvStringIO = StringIO(fresult)
df = pd.read_csv(csvStringIO, sep=",", error_bad_lines=False)

C:\Users\isaac\AppData\Local\Temp\ipykernel_17704\1000733840.py:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(csvStringIO, sep=",", error_bad_lines=False)


In [1]:
#Examining the table on the website, I noticed there was a column for Overtime data which was not used in all cases. 
#In order for the column headers to align properly, we split the Overtime and Non-overtime rows and merge them into an 
#updated dataframe later.

OT = df.loc[224:len(df)]

NameError: name 'df' is not defined

In [5]:
#Non-Overtime split.

no_OT = df.loc[0:223]

,Team Name,Year,Wins,Losses,OT Losses,Win%,Goals For,Goals Against,+/-
0,Boston Bruins,1990,44,24,0.550,299.0,264,35,NaN
1,Buffalo Sabres,1990,31,30,0.388,292.0,278,14,NaN
2,Calgary Flames,1990,46,26,0.575,344.0,263,81,NaN
3,Chicago Blackhawks,1990,49,23,0.613,284.0,211,73,NaN
4,Detroit Red Wings,1990,34,38,0.425,273.0,298,-25,NaN
...,...,...,...,...,...,...,...,...,...
219,St. Louis Blues,1998,37,32,0.451,237.0,209,28,NaN
220,Tampa Bay Lightning,1998,19,54,0.232,179.0,292,-113,NaN
221,Toronto Maple Leafs,1998,45,30,0.549,268.0,231,37,NaN
222,Vancouver Canucks,1998,23,47,0.280,192.0,258,-66,NaN


In [6]:
#Renaming the Non-Overtime columns to appropriate column headers for merge.
no_OT = no_OT.rename(columns = {
             'OT Losses' : 'Win%',
             'Win%':'Goals For',
             'Goals For' : 'Goals Against',
             'Goals Against' : '+/-',
             '+/-' : 'remove'
})

In [7]:
#Adding column after 4th column to add "N/A" to OT Losses to the non-overtime for merge.
no_OT.insert(4,'OT Losses',"N/A")

In [8]:
#Merging the two split dataframes to a unified dataframe.
frames = [no_OT, OT]
result = pd.concat(frames)
result = result.drop('remove',axis=1)

In [10]:
#Verifying structure by inspecting top 5 rows.
result.head()

,Team Name,Year,Wins,Losses,OT Losses,Win%,Goals For,Goals Against,+/-,Total Games,Total Goals,AVG Goals per Game
0,Boston Bruins,1990,44,24,N/A,0.550,299.0,264,35.0,68,563.0,8.279412
1,Buffalo Sabres,1990,31,30,N/A,0.388,292.0,278,14.0,61,570.0,9.344262
2,Calgary Flames,1990,46,26,N/A,0.575,344.0,263,81.0,72,607.0,8.430556
3,Chicago Blackhawks,1990,49,23,N/A,0.613,284.0,211,73.0,72,495.0,6.875000
4,Detroit Red Wings,1990,34,38,N/A,0.425,273.0,298,-25.0,72,571.0,7.930556


In [ ]:
#Exporting resulting table into a CSV File for manipulation in Tableau
filepath = Path('D:/Py4e/Beautiful Soup Tutorials/Scrape This Site/ScrapeThisSite NHL TEams/teams.csv')
filepath.parent.mkdir(parents=True,exist_ok=True)
result.to_csv(filepath)
